## Production and Transportation Scheduling

### Problem Description:

- Assign a given set of orders to plants and carriers to minimize cost.
    - select the appropriate plant that processes the order
    - select the port from which to send the order to its destination
- Restrictions
    - a given plant can only process certain products
    - each plant is only connected to certain ports. (see Supply Chain Mapping below.)
    - some customers have a vendor managed inventory (VMI) which means that they can only be serviced by a certain plant.

## Code Implementation

### Reading in Data and Graphs

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

In [14]:
# reading in all the csv files
plant_ports = pd.read_csv("PlantPorts.csv", index_col=0)
order_list = pd.read_csv("OrderList.csv", index_col=0)
products_plants = pd.read_csv("ProductsPerPlant.csv", index_col=0)
vmi_plants = pd.read_csv("VmiCustomers.csv", index_col=0)
freight_rates = pd.read_csv("FreightRates.csv", index_col=0)
wh_cost = pd.read_csv("WhCosts.csv", index_col=0)

# changing column names
order_list.columns = [i.replace(" ", "_") for i in order_list.columns]
products_plants.columns = [i.replace(" ", "_") for i in products_plants.columns]
plant_ports.columns = [i.replace(" ", "_") for i in plant_ports.columns]
vmi_plants.columns = [i.replace(" ", "_") for i in vmi_plants.columns]

wh_cost.set_index("WH", inplace=True)

In [15]:
plant_ports_graph = nx.from_pandas_edgelist(plant_ports, source="Plant_Code", target="Port")

### Supply Chain Mapping

#### Non Interactive with Networkx

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.set_facecolor("Grey")

# specify layout for the graph
# layout = nx.bipartite_layout(plant_ports_graph, plant_ports["Plant Code"])

layout = nx.bipartite_layout(plant_ports_graph, plant_ports["Plant_Code"])

for i in layout:
    if i.startswith("PLANT"):
        layout[i][0] -= 0.1
    else:
        layout[i][0] += 0.1

# we want to map the degree of the node to a color/size
degrees = dict(plant_ports_graph.degree)
maps = [v*100 for v in degrees.values()]

# specify the color map
cmap = plt.cm.Blues

# keyword args that are the same for both functions
kwargs = {"pos":layout, "ax":ax}

nx.draw_networkx_nodes(plant_ports_graph, node_size=maps, node_color=maps, cmap=cmap, **kwargs)
nx.draw_networkx_edges(plant_ports_graph, **kwargs)
nx.draw_networkx_labels(plant_ports_graph, pos=layout)
plt.show()

#### Interactive Version with Plotly.

- code adopted from https://plotly.com/python/network-graphs/

In [17]:
# get starting and ending points of the edges and add them to the graph
layout = nx.bipartite_layout(plant_ports_graph, plant_ports["Plant_Code"])

for i in layout:
    if i.startswith("PLANT"):
        layout[i][0] += 0.3
    else:
        layout[i][0] -= 0.3

edge_x = []
edge_y = []

for edge in plant_ports_graph.edges():
    x0, y0 = layout[edge[0]]
    x1, y1 = layout[edge[1]]
    
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#25488e'),
    hoverinfo='none',
    mode='lines')

In [18]:
# get coordinated of nodes and add them to the graph

node_x = []
node_y = []
for node in plant_ports_graph.nodes():
    x, y = layout[node]
    node_x.append(x)
    node_y.append(y)
    
maps = [v for v in degrees.values()]

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=maps,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# node_trace.text = ["Number of Links: " + str(i) for i in maps]
node_trace.text = [i + " Number of Links: " + str(degrees[i]) for i in degrees]

In [36]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Supply Chain',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[dict(text="Factories",
                     showarrow=False,
                     xref="paper", yref="paper",
                     x=0.005, y=-0.002 ),
                             dict(text="Ports",
                     showarrow=False,
                     xref="paper", yref="paper",
                     x=0.95, y=-0.002 )],
                xaxis=dict(showgrid=True, zeroline=False, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=False, showticklabels=True))
                )
fig

### Preparing the Order Table

In [4]:
order_new = order_list.drop(columns=["Order_Date", "Origin_Port", "Carrier", "Plant_Code", "TPT", "Service_Level", "Ship_ahead_day_count", "Ship_Late_Day_count"])
order_new.set_index("Order_ID", inplace=True)
order_new.to_csv("order_new.csv")

### Preparing Freight Table

In [5]:
freight_rates.drop(columns=["dest_port_cd", "Carrier type", "svc_cd"], inplace=True)
freight_rates.to_csv("FreightRates_mod.csv")

### Problem Restrictions

In [6]:
# given a product id, return the plants that can produce this product.
def product_restriction(index):
    data = order_new.loc[index]
    product_id = data["Product_ID"]
    possible_plants = products_plants.loc[products_plants["Product_ID"] == product_id]
    return np.array(possible_plants["Plant_Code"])

In [7]:
# vmi restriction: check if a given customer has to be serviced by a specific facility, else return all facilities as possibilities.
def customer_restriction(index):
    data = order_new.loc[index]
    Customer_id = data["Customer"]
    possible_plants = vmi_plants.loc[vmi_plants["Customers"] == Customer_id]
    if list(possible_plants["Plant_Code"]) == []:
        return plant_ports["Plant_Code"].unique()
    else:
        return np.array(possible_plants["Plant_Code"])

In [8]:
# combine both the product and vmi restriction. There will be orders for which only one, or possibly even 0, facilities can fullfil it.
def check_order(Order_Id, length=True):
    if length:
        return len(np.intersect1d(customer_restriction(Order_Id), product_restriction(Order_Id)))
    else:
        return np.intersect1d(customer_restriction(Order_Id), product_restriction(Order_Id))

In [9]:
# under the restrictions above, we can calculate the number of facilities that can process a given order.
order_new["decision_space_size"] = np.array(list(map(check_order, order_new.index)))

In [10]:
order_new["decision_space_size"].value_counts()

1    6275
0    1045
4     982
2     785
3     127
5       1
Name: decision_space_size, dtype: int64

We can see that for most orders, there is only one facility that can handle the order. For ~1,000 there is no possible facility that can handle the order given our problem restrictions, we will exclude these orders from our further optimization problem.

### Assigning orders

In [11]:
# we will exclude orders that cannot be processed by any facility (i.e. where the decision space size == 0)
order_new = order_new.loc[order_new.decision_space_size != 0]

In [12]:
# return the possible plants that can process the order.
order_new["decision_space"] = np.array((map(lambda x: check_order(x, length=False), order_new.index)))

In [ ]:
order_new.head(3)

In [21]:
dec_space = ["PLANT16", "PLANT02"]

In [ ]:
order_new["decision_space"].apply(lambda x: )

In [26]:
def min_cost(dec_space):
    possible_plants = wh_cost.loc[dec_space]
    return possible_plants.loc[possible_plants["Cost/unit"] == min(possible_plants["Cost/unit"])]

In [21]:
ports_agg = freight_rates.groupby(["orig_port_cd"]).agg(avg_rate=("rate", np.mean))

In [31]:
ports_agg

,avg_rate
orig_port_cd,
PORT02,1.874696
PORT03,9.979378
PORT04,1.940713
PORT05,2.872483
PORT06,2.532242
PORT07,0.197100
PORT08,0.592297
PORT09,2.836567
PORT10,5.368383


In [34]:
# given a port and the order specifications return the carrier that can handle the product at the best price.
def best_port(plant_id):
    
    # weight = order_one.loc[order_id]["Weight"]
    # carriers = freight_rates.loc[(freight_rates.max_wgh_qty > weight)]
    # carriers = freight_rates.loc[(freight_rates.orig_port_cd == port_id) & (freight_rates.max_wgh_qty > weight) & (freight_rates.minm_wgh_qty < weight)]
    possible_ports = plant_ports.loc[(plant_ports.Plant_Code == plant_id) & (plant_ports.Port != "PORT01"), "Port"]
    return ports_agg.loc[possible_ports]
    

In [35]:
best_port("PLANT01")

,avg_rate
orig_port_cd,
PORT02,1.874696


In [ ]:
freight_rates
sns.histplot(data=freight_rates, x="max_wgh_qty")

In [ ]:
freight_rates.describe()

Most facilities have only one connection to a port. Port 4 is potentially the most important one as it has the most connections to the warehouses.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
plt.xticks(rotation=45)
plant_counts = pd.DataFrame(products_plants["Plant_Code"].value_counts())
ax.bar(plant_counts.index, plant_counts["Plant_Code"])

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
plt.xticks(rotation=45)
ax.bar(wh_cost.WH, wh_cost["Cost/unit"])

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
plt.xticks(rotation=45)
plant_counts = pd.DataFrame(products_plants["Plant_Code"].value_counts())
sns.barplot(x=plant_counts.index,y=plant_counts["Plant_Code"], ax=ax)

In [ ]:
wh_cost.set_index("WH", inplace=True)

In [ ]:
plant_info = pd.concat([wh_cost, plant_counts], axis=1)

In [ ]:
x = np.arange(len(plant_info.index))
width = 0.35
fig, ax1 = plt.subplots(figsize=(10,10))
plt.xticks(rotation=45)
ax1.bar(x - width/2, plant_info["Cost/unit"], width, label='Cost/unit', color="blue")
ax1.set_ylabel("Cost/unit")

ax2 = ax1.twinx()
ax2.bar(x + width/2, plant_info["Plant_Code"], width, label='Capacity', color="red")
ax2.set_ylabel("Capacity")

ax1.set_xticks(x)
ax1.set_xticklabels(list(plant_info.index))
plt.show()

In [ ]:
def get(elements):
    to_return = 0
    for value in reversed(elements):
        if value % 2 != 0:
            print(True)
            to_return = value
    return to_return
    
get([7,3,4,5])

True
True
True


7

In [1]:
def sum_numbers(start=1, end=10):
    result = 0
    for i in range(start, end):
        result += i
    return result

a = 10
b = 10

sum_numbers(0,4)

6

In [32]:
[0,1,2,3].append()

[4, 3, 2]

In [ ]:
sorted(505)

In [ ]:
sorted(501, reverse=True)

In [ ]:
liste = [("Haus", "kunde"),("AB", "UM"),("ich", "aber")]

list(map(lambda y: [i + "1" for i in y], liste))

In [ ]:
for i in liste:
    print(i + "1")

In [42]:
["Haus", "kunde"]


'Haus1kunde'